<a href="https://colab.research.google.com/github/wFairmanOlin/HAUCS/blob/main/GPS_plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates 
import pandas as pd
from datetime import datetime
from datetime import timedelta
import firebase_admin
from firebase_admin import db
from firebase_admin import credentials
import plotly.express as px

In [47]:
### ONLY RUN ONCE! ###
# #download data from firebase
# cred = credentials.Certificate("/content/haucs-monitoring-firebase-adminsdk-wt1pb-1291f2083e.json")
# firebase_admin.initialize_app(cred, {'databaseURL': 'https://haucs-monitoring-default-rtdb.firebaseio.com'})
try:
    print(cred)
except:
    cred = credentials.Certificate("../Biomass/pc_basestation/fb_key.json")
    firebase_admin.initialize_app(cred, {'databaseURL': 'https://haucs-monitoring-default-rtdb.firebaseio.com'})

In [48]:
## ONLY RUN TO REFRESH DATA! ###
ref = db.reference('/LH_Farm')
data = ref.get()

### RUN TO SAVE JSON FILE LOCALLY ###
with open('latest_data.json', 'w') as f:
    json.dump(data, f)

In [49]:
gps_df = pd.DataFrame(data['gps'], index=['RSSI', 'lat', 'lng']).T.reset_index()
gps_df.columns = ['timestamp', 'RSSI', 'lat', 'lng']

gps_df.head()

,timestamp,RSSI,lat,lng
0,20230914_17:47:26,-55,27.535631,-80.351898
1,20230914_17:47:31,-54,27.535633,-80.351898
2,20230914_17:47:36,-55,27.535633,-80.351898
3,20230914_17:47:41,-55,27.535634,-80.351890
4,20230914_17:47:46,-57,27.535638,-80.351890


In [50]:
# Split the concatenated values in the lat and lng columns
gps_df[['lat', 'lng']] = gps_df[['lat', 'lng']].apply(lambda x: x.str.split().str[0])

# Convert the necessary columns to numeric type
gps_df['RSSI'] = pd.to_numeric(gps_df['RSSI'])
gps_df['lat'] = pd.to_numeric(gps_df['lat'])
gps_df['lng'] = pd.to_numeric(gps_df['lng'])

In [51]:
# Convert the timestamp column to datetime type
gps_df['timestamp'] = pd.to_datetime(gps_df['timestamp'], format='%Y%m%d_%H:%M:%S')

gps_df = gps_df[gps_df['timestamp'] > (datetime(year=2023, month=9, day=20) - timedelta(days=5))]

gps_df.head()

# # Create the geographical heatmap plot using Plotly Express
# fig = px.density_mapbox(gps_df, lat='lat', lon='lng', z='RSSI', radius=10, mapbox_style='carto-positron')

# # Set mapbox token and layout
# fig.update_layout(
#     mapbox=dict(
#         accesstoken='YOUR_MAPBOX_TOKEN',
#         center=dict(lat=gps_df['lat'].mean(), lon=gps_df['lng'].mean()),
#         zoom=10
#     )
# )

# # Display the plot
# fig.show()

,timestamp,RSSI,lat,lng
39,2023-09-16 14:57:35,-76,37.706920,-89.468360
40,2023-09-16 14:58:17,-93,37.706844,-89.465377
41,2023-09-16 14:58:22,-89,37.706855,-89.465011
42,2023-09-16 14:58:27,-93,37.706821,-89.464828
43,2023-09-16 14:58:32,-96,37.706806,-89.464584


In [57]:
# px.set_mapbox_access_token(open(".mapbox_token").read())

fig = px.scatter_mapbox(gps_df,
                        lat="lat",
                        lon="lng",
                        hover_name="RSSI",
                        color="RSSI",
                        mapbox_style='carto-positron',
                        zoom=1)

# Set mapbox token and layout
fig.update_layout(
    width=800,
    height=500,
    mapbox=dict(
        accesstoken='YOUR_MAPBOX_TOKEN',
        style='open-street-map',
        center=dict(lat=gps_df['lat'].mean(), lon=gps_df['lng'].mean()),
        zoom=14,
        
    )
)


In [53]:
#select sensor and data or status path
# del_path = '/LH_Farm/pond_71'
# # del_path = '/LH_Farm'
# del_node = db.reference(del_path)
# #select start date for delete 
# del_start = datetime(2023, 5, 23, hour=8, minute=15)
# #select end date for delete
# del_end = datetime(2023, 9, 10, hour=20, minute=0)

# del_data = del_node.get()
# del_data_dt = []
# print("Deleting the Following Nodes: ", del_path)
# for i in del_data:
#     i_dt = datetime.fromisoformat(i)
#     if (del_start <= i_dt <= del_end):
#         print(i)
#         del_data_dt.append(i)
        
#uncomment the following section to delete
# print("Should these nodes be deleted? (y/n)")
# if (input('permanently delete (y/n): ') == 'y'):
# for i in del_data_dt:
#     del_point = del_path + '/' + i
#     del_point = db.reference(del_point)
#     del_point.delete()